In [1]:
import pandas as pd 
import pulp

In [31]:
# Read in projections
proj = pd.read_csv(r"C:\Users\james\Documents\MLB\Data\Y. One Time Use\Steamer Weekly Log\Batters_FG_20230327.csv")

# Create Name for merge
proj['Name'] = proj['firstname'] + " " + proj['lastname']
# Keep relevant variable
proj = proj[['Name', 'HR']]
# Rename to distinguish from last year's HR
proj.rename(columns={'HR':'proj'}, inplace=True)

# Read in contest sheet
df = pd.read_excel(r"C:\Users\james\Documents\MLB\Data\Other\HRD - Updated.xlsx", sheet_name='22 Stats')

# Remove weird spaces that aren't actually spaces
df['Name'] = df['Name'].replace(r'\xa0', " ", regex=True)

# Merge
df = df.merge(proj, on=['Name'], how='inner')
df.drop_duplicates('Name', inplace=True)
df = df[['Name', 'HR', 'proj']]
player_dict = df.set_index('Name').to_dict('index')
player_dict = {k.replace(' ', '_'): v for k, v in player_dict.items()}
player_dict

{'Aaron_Judge': {'HR': 62, 'proj': 44.2097},
 'Kyle_Schwarber': {'HR': 46, 'proj': 38.58},
 'Mike_Trout': {'HR': 40, 'proj': 39.4108},
 'Pete_Alonso': {'HR': 40, 'proj': 38.0539},
 'Austin_Riley': {'HR': 38, 'proj': 33.9163},
 'Yordan_Alvarez': {'HR': 37, 'proj': 33.6672},
 'Christian_Walker': {'HR': 36, 'proj': 27.4884},
 'Mookie_Betts': {'HR': 35, 'proj': 30.819},
 'Paul_Goldschmidt': {'HR': 35, 'proj': 29.1058},
 'Rowdy_Tellez': {'HR': 35, 'proj': 30.4919},
 'Shohei_Ohtani': {'HR': 34, 'proj': 34.6484},
 'Matt_Olson': {'HR': 34, 'proj': 34.2097},
 'Anthony_Santander': {'HR': 33, 'proj': 29.1019},
 'Corey_Seager': {'HR': 33, 'proj': 26.5549},
 'Vladimir_Guerrero_Jr.': {'HR': 32, 'proj': 38.8337},
 'Manny_Machado': {'HR': 32, 'proj': 29.7804},
 'Anthony_Rizzo': {'HR': 32, 'proj': 26.5916},
 'Giancarlo_Stanton': {'HR': 31, 'proj': 32.4617},
 'Willy_Adames': {'HR': 31, 'proj': 27.2288},
 'Eugenio_Suarez': {'HR': 31, 'proj': 26.663},
 'Kyle_Tucker': {'HR': 30, 'proj': 33.9589},
 'Nolan_A

In [34]:
from pulp import *

# create the LP object, set up as a maximization problem
lp = pulp.LpProblem('Home Run Maximization', LpMaximize)

x = pulp.LpVariable.dicts('x', player_dict.keys(), lowBound=0, cat='Binary')

# create the constraint for the sum of prior year's home runs
lp += sum(player_dict[player]['HR'] * x[player] for player in player_dict) <= 168

# create the constraint for the number of players in the lineup
lp += sum(x[player] for player in player_dict) == 8

# create the objective function to maximize
lp += sum(player_dict[player]['proj'] * x[player] for player in player_dict)

# solve the LP
lp.solve()

# print the solution
print('Projected number of home runs for each player in the lineup:')
for player in player_dict:
    if x[player].varValue == 1:
        print(player, ':', player_dict[player]['proj'])

print('Total projected number of home runs:', pulp.value(lp.objective))



Projected number of home runs for each player in the lineup:
Mike_Trout : 39.4108
Vladimir_Guerrero_Jr. : 38.8337
Kyle_Tucker : 33.9589
Eloy_Jimenez : 30.9697
Ronald_Acuna_Jr. : 30.1026
Jazz_Chisholm_Jr. : 25.9945
Joey_Meneses : 24.5467
Brandon_Lowe : 24.9707
Total projected number of home runs: 248.78759999999997
